### Import Libraries

In [ ]:
import sys
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

### Load SCADA Data

In [ ]:
# Input the base directory where Kelmarsh Dataset is located
# base_directory = input("Enter the base directory for Kelmarsh Dataset: ")

# Create file paths for the four Turbine_Data_Kelmarsh data files
file_paths = [
    f"{base_directory}/Kelmarsh_SCADA_2018_3084/Turbine_Data_Kelmarsh_1_2018-01-01_-_2019-01-01_228.csv",
    f"{base_directory}/Kelmarsh_SCADA_2019_3085/Turbine_Data_Kelmarsh_1_2019-01-01_-_2020-01-01_228.csv",
    f"{base_directory}/Kelmarsh_SCADA_2020_3086/Turbine_Data_Kelmarsh_1_2020-01-01_-_2021-01-01_228.csv",
    f"{base_directory}/Kelmarsh_SCADA_2021_3087/Turbine_Data_Kelmarsh_1_2021-01-01_-_2021-07-01_228.csv"
]

# Initialize an empty list to store the dataframes
dataframes = []

# Read the data from the files and store in dataframes
for file_path in file_paths:
    WT_data = pd.read_csv(file_path, sep=',', skiprows=9)
    dataframes.append(WT_data)

# Concatenate all dataframes into one
WT_data = pd.concat(dataframes)
WT_data.head()

### Data Visualization

In [ ]:
WT_data = WT_data.drop_duplicates()
WT_data = WT_data.groupby(['Wind speed (m/s)', 'Power (kW)']).mean().reset_index()

# Plotting
plt.figure(figsize=(10, 6))

# Plotting Wind speed vs Power
plt.subplot(2, 2, 1)
sns.scatterplot(data=WT_data, x='Wind speed (m/s)', y='Power (kW)')
plt.title('Wind speed vs Power')

# Plotting Rotor speed vs Power
plt.subplot(2, 2, 2)
sns.scatterplot(data=WT_data, x='Rotor speed (RPM)', y='Power (kW)')
plt.title('Rotor speed vs Power')

# Plotting Wind speed vs Rotor speed
plt.subplot(2, 2, 3)
sns.scatterplot(data=WT_data, x='Wind speed (m/s)', y='Rotor speed (RPM)')
plt.title('Wind speed vs Rotor speed')

plt.tight_layout()
plt.show()

### Data Preperation

In [ ]:
# Removing any rows with missing values
# Selecting the columns to filter based on a regular expression pattern
data_filter = WT_data.filter(regex='(count|Density|Index|Energy|Equivalent|Production|Cable|Voltage|factor|Current|Default|Virtual|Potential|potential|Avail|IEC|Lost|Max|Min|min|max|Std|Maximum|Minimum|deviation)')

# Dropping the filtered columns and remove rows with missing values
WT_data = WT_data.drop(data_filter.columns, axis=1)
WT_data = WT_data.dropna(axis=0)

# Converts the '# Date and time' column to datetime format
WT_data['# Date and time'] = pd.to_datetime(WT_data['# Date and time'],format='%Y-%m-%d %H:%M:%S')

# Sort the dataset based on the '# Date and time' column
WT_data = WT_data.sort_values('# Date and time')

# Reset the index of the dataset
WT_data.reset_index(drop=True, inplace=True)

### Label The Faults in The Data

In [ ]:
# Combining Kelmarsh Turbine Status Data from multiple years
file_paths = [
    f"{base_directory}/Kelmarsh_SCADA_2018_3084/Status_Kelmarsh_1_2018-01-01_-_2019-01-01_228.csv",
    f"{base_directory}/Kelmarsh_SCADA_2019_3085/Status_Kelmarsh_1_2019-01-01_-_2020-01-01_228.csv",
    f"{base_directory}/Kelmarsh_SCADA_2020_3086/Status_Kelmarsh_1_2020-01-01_-_2021-01-01_228.csv",
    f"{base_directory}/Kelmarsh_SCADA_2021_3087/Status_Kelmarsh_1_2021-01-01_-_2021-07-01_228.csv"
]

dataframes = []

for file_path in file_paths:
    data_status = pd.read_csv(file_path, sep=',', skiprows=9)
    # Removing invalid duration rows
    data_status = data_status.drop(data_status[data_status['Duration'] == '-'].index)
    # Reset the index of the dataset
    data_status.reset_index(drop=True, inplace=True)
    dataframes.append(data_status)

data_status = pd.concat(dataframes)

# Converts the 'Timestamp start' and 'Timestamp end' columns to datetime format
data_status['Timestamp start'] = pd.to_datetime(data_status['Timestamp start'],format='%Y-%m-%d %H:%M:%S')
data_status['Timestamp end'] = pd.to_datetime(data_status['Timestamp end'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
data_status.head()

In [ ]:
# Collecting fault periods
fault_periods = []
for i, row in data_status.iterrows():
    start = row['Timestamp start']
    end = row['Timestamp end']
    fault_periods.append((start, end))

# Generating anomaly labels based on fault periods
anomaly_labels = []
for i, row in WT_data.iterrows():
    timestamp = pd.to_datetime(row['# Date and time'],format='%Y-%m-%d %H:%M:%S')
    anomaly = 0
    for start, end in fault_periods:
        if start <= timestamp <= end:
            anomaly = 1
            break
    anomaly_labels.append(anomaly)
    
# Converting the anomaly_classes list into a NumPy array
anomaly_labels = np.array(anomaly_labels)

print('Number of faults in the dataset :',anomaly_labels.sum())

In [ ]:
WT_data = WT_data[['# Date and time', 'Rotor speed (RPM)', 'Wind speed (m/s)', 'Power (kW)']]
WT_data['fault'] = anomaly_labels
WT_data.head()

In [ ]:
sns.relplot(x='Wind speed (m/s)', y='Power (kW)', hue=anomaly_labels, data=WT_data)

In [ ]:
sns.relplot(x='Wind speed (m/s)', y='Rotor speed (RPM)', hue=anomaly_labels, data=WT_data)

### Data Fitting

In [ ]:
from pygam import GAM

# remove data with faults
data_fit = WT_data[WT_data['fault']==0].copy()

# Extract the relevant columns from the DataFrame
wind_speed = data_fit['Wind speed (m/s)'].values
power = data_fit['Power (kW)'].values
rotor_speed = data_fit['Rotor speed (RPM)'].values

# Fit a GAM for power
gam_power = GAM(n_splines=20).fit(wind_speed, power)

# Fit a GAM for rotor speed
gam_rotor = GAM(n_splines=20).fit(wind_speed, rotor_speed)

# Generate points for the fitted curves
wind_speed_fit = np.linspace(wind_speed.min(), wind_speed.max(), 100)
power_fit = gam_power.predict(wind_speed_fit)
rotor_speed_fit = gam_rotor.predict(wind_speed_fit)

# Plot the results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.scatter(wind_speed, power, label='Data')
ax1.plot(wind_speed_fit, power_fit, color='red', label='GAM Fit')
ax1.set_xlabel('Wind Speed (m/s)', fontsize=14)
ax1.set_ylabel('Power (kW)', fontsize=14)
ax1.set_title('Wind Speed vs Power', fontsize=16)
ax1.legend(fontsize=12)
ax1.tick_params(axis='both', labelsize=12)

ax2.scatter(wind_speed, rotor_speed, label='Data')
ax2.plot(wind_speed_fit, rotor_speed_fit, color='red', label='GAM Fit')
ax2.set_xlabel('Wind Speed (m/s)', fontsize=14)
ax2.set_ylabel('Rotor Speed (RPM)', fontsize=14)
ax2.set_title('Wind Speed vs Rotor Speed', fontsize=16)
ax2.legend(fontsize=12)
ax2.tick_params(axis='both', labelsize=12)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predict power and rotor speed using the fitted GAMs
power_pred = gam_power.predict(wind_speed)
rotor_speed_pred = gam_rotor.predict(wind_speed)

# Calculate Mean Absolute Error (MAE)
power_mae = mean_absolute_error(power, power_pred)
rotor_speed_mae = mean_absolute_error(rotor_speed, rotor_speed_pred)

# Calculate Root Mean Squared Error (RMSE)
power_rmse = np.sqrt(mean_squared_error(power, power_pred))
rotor_speed_rmse = np.sqrt(mean_squared_error(rotor_speed, rotor_speed_pred))

power_max_error = np.max(np.abs(power - power_pred))
rotor_speed_max_error = np.max(np.abs(rotor_speed - rotor_speed_pred))

print(f"Power - Mean Absolute Error (MAE): {power_mae:.2f} kW")
print(f"Power - Root Mean Squared Error (RMSE): {power_rmse:.2f} kW")
print(f"Power - Maximum Error: {power_max_error:.2f} kW")
print(f"Rotor Speed - Mean Absolute Error (MAE): {rotor_speed_mae:.2f} RPM")
print(f"Rotor Speed - Root Mean Squared Error (RMSE): {rotor_speed_rmse:.2f} RPM")
print(f"Rotor Speed - Maximum Error: {rotor_speed_max_error:.2f} RPM")

In [ ]:
plt.hist(abs(power_pred-power), bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Power (kW)')
plt.ylabel('Frequency')
plt.title('Distribution of |Actual - Predicted| Power')
plt.show()

In [ ]:
plt.hist(abs(rotor_speed_pred-rotor_speed), bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Rotor Speed (RPM)')
plt.ylabel('Frequency')
plt.title('Distribution of |Actual - Predicted| Rotor Speed')
plt.show()

### Simulating Cyberattacks

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
import random

np.random.seed(50)
random.seed(50)

def truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def single_parameter_manipulation(data, column, start_index, duration, mean_error=0.7, error_range=(0.5, 0.9)):
    end_index = start_index + duration
    error_dist = truncated_normal(mean_error, 0.1, error_range[0], error_range[1])
    error_factor = error_dist.rvs(duration + 1)
    data.loc[start_index:end_index, column] = data.loc[start_index:end_index, column] * (1 + error_factor)
    data.loc[start_index:end_index, 'Attack'] = 1
    
    return data, start_index, end_index

def multiple_parameter_manipulation(data, columns, start_index, duration, mean_error=0.7, error_range=(0.5, 0.9)):
    end_index = start_index + duration
    for column in columns:
        data, _, _ = single_parameter_manipulation(data, column, start_index, duration, mean_error, error_range)
    return data, start_index, end_index

def data_repetition(data, start_index, duration):
    end_index = start_index + duration
    repetition_start = start_index - duration - 1
    repetition_end = start_index - 1
    data.loc[start_index:end_index] = data.loc[repetition_start:repetition_end].values
    data.loc[start_index:end_index, 'Attack'] = 1
    return data, start_index, end_index

def simulated_fault_type_1(data, column, start_index, duration):
    end_index = start_index + duration
    data.loc[start_index:end_index, column] = 0
    data.loc[start_index:end_index, 'Attack'] = 1
    return data, start_index, end_index

def simulated_fault_type_2(data, column, start_index, duration, sd=0.5):
    end_index = start_index + duration
    error_dist = np.random.normal(0, sd, duration + 1)
    data.loc[start_index:end_index, column] = data.loc[start_index:end_index, column] * (1 + error_dist)
    data.loc[start_index:end_index, 'Attack'] = 1
    return data, start_index, end_index

def simulate_attacks(data, num_attacks=100, min_duration=6, max_duration=24, min_duration_fault=2, max_duration_fault=6):
    manipulated_data = data.copy()
    manipulated_data['Attack'] = 0
    manipulated_data['Attack_Type'] = 0

    attack_info_list = []
    attack_number = 1

    for attack_num in range(num_attacks):
        print(f"Attack {attack_num + 1}")

        attack_type = random.randint(1, 4) 
        print(f"Attack Type: {attack_type}")

        if attack_type == 1:  # Single Parameter Manipulation
            parameter = random.choice(['Rotor speed (RPM)', 'Wind speed (m/s)', 'Power (kW)'])
            start_index = random.randint(0, len(manipulated_data) - max_duration)
            duration = random.randint(min_duration, max_duration)
            manipulated_data.loc[start_index:start_index + duration, 'Attack_Type'] = 1
            manipulated_data, start_single, end_single = single_parameter_manipulation(manipulated_data, parameter, start_index, duration)
            attack_info_list.append({
                'Attack_Number': attack_number,
                'Starting_Time': start_single,
                'End_Time': end_single
            })
            attack_number += 1

        elif attack_type == 2:  # Multiple Parameter Manipulation
            num_parameters = random.randint(2, 3)
            parameters = random.sample(['Rotor speed (RPM)', 'Wind speed (m/s)', 'Power (kW)'], k=num_parameters)
            start_index = random.randint(0, len(manipulated_data) - max_duration)
            duration = random.randint(min_duration, max_duration)
            manipulated_data.loc[start_index:start_index + duration, 'Attack_Type'] = 2
            manipulated_data, start_multiple, end_multiple = multiple_parameter_manipulation(manipulated_data, parameters, start_index, duration)
            attack_info_list.append({
                'Attack_Number': attack_number,
                'Starting_Time': start_multiple,
                'End_Time': end_multiple
            })
            attack_number += 1

        elif attack_type == 3:  # Data Repetition
            start_index = random.randint(max_duration, len(manipulated_data) - max_duration)  # Ensure enough data for repetition
            duration = random.randint(min_duration, max_duration)
            manipulated_data, start_repetition, end_repetition = data_repetition(manipulated_data, start_index, duration)
            attack_info_list.append({
                'Attack_Number': attack_number,
                'Starting_Time': start_repetition,
                'End_Time': end_repetition
            })
            manipulated_data.loc[start_index:start_index + duration, 'Attack_Type'] = 3
            attack_number += 1

        elif attack_type == 4:  # Simulated Fault Type 1
            parameter = random.choice(['Rotor speed (RPM)', 'Wind speed (m/s)', 'Power (kW)'])
            start_index = random.randint(0, len(manipulated_data) - max_duration_fault)
            duration = random.randint(min_duration_fault, max_duration_fault)
            manipulated_data.loc[start_index:start_index + duration, 'Attack_Type'] = 4
            manipulated_data, start_fault1, end_fault1 = simulated_fault_type_1(manipulated_data, parameter, start_index, duration)
            attack_info_list.append({
                'Attack_Number': attack_number,
                'Starting_Time': start_fault1,
                'End_Time': end_fault1
            })
            attack_number += 1


    # Convert list to DataFrame
    attack_info = pd.DataFrame(attack_info_list)

    return manipulated_data, attack_info

# Perform cyber attacks
min_duration = 6  # Minimum duration of 1 hour (assuming data is recorded every 10 minutes)
max_duration = 24  # Maximum duration of 4 hours (assuming data is recorded every 10 minutes)
min_duration_fault = 2  # Minimum duration of 20 minutes
max_duration_fault = 6  # Maximum duration of 1 hour

manipulated_data, attack_info = simulate_attacks(WT_data, num_attacks=100, min_duration=min_duration, max_duration=max_duration,
                                                 min_duration_fault=min_duration_fault, max_duration_fault=max_duration_fault)

In [ ]:
manipulated_data.head()

In [ ]:
manipulated_data['Power_GAM'] = gam_power.predict(manipulated_data['Wind speed (m/s)'])
manipulated_data['Rotor_Speed_GAM'] = gam_rotor.predict(manipulated_data['Wind speed (m/s)'])
manipulated_data[manipulated_data['Attack']==1].head()

In [ ]:
# Remove rows with 'fault' equal to 1 and 'Power (kW)' less than 0
manipulated_data_2 = manipulated_data[(manipulated_data['fault'] != 1) & (manipulated_data['Power (kW)'] >= 0)]

# Reset index if you want consecutive integer indices
manipulated_data_2.reset_index(drop=True, inplace=True)

# Calculate the absolute differences for Power and Rotor Speed
manipulated_data_2['Power_GAM_diff'] = abs(manipulated_data_2['Power (kW)'] - manipulated_data_2['Power_GAM'])
manipulated_data_2['Rotor_Speed_GAM_diff'] = abs(manipulated_data_2['Rotor speed (RPM)'] - manipulated_data_2['Rotor_Speed_GAM'])

manipulated_data_2.head()

## LSTM with GAM

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import h5py

# Split the data into training and testing sets
train_data = manipulated_data_2[manipulated_data_2['# Date and time'].dt.year < 2021]
test_data = manipulated_data_2[manipulated_data_2['# Date and time'].dt.year == 2021]

# Define the input features and target
input_features = ['Wind speed (m/s)', 'Power_GAM_diff', 'Rotor_Speed_GAM_diff']
target = 'Attack'

# Prepare the training and testing data
X_train = train_data[input_features].values
y_train = train_data[target].values
X_test = test_data[input_features].values
y_test = test_data[target].values

# Scale the input features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the input data to include the sequence length
sequence_length = 10
X_train_reshaped = []
y_train_reshaped = []
for i in range(sequence_length, len(X_train_scaled)):
    X_train_reshaped.append(X_train_scaled[i - sequence_length:i])
    y_train_reshaped.append(y_train[i])  # Shift the labels by sequence_length
X_train_reshaped = np.array(X_train_reshaped)
y_train_reshaped = np.array(y_train_reshaped)

X_test_reshaped = []
y_test_reshaped = []
for i in range(sequence_length, len(X_test_scaled)):
    X_test_reshaped.append(X_test_scaled[i - sequence_length:i])
    y_test_reshaped.append(y_test[i])  # Shift the labels by sequence_length
X_test_reshaped = np.array(X_test_reshaped)
y_test_reshaped = np.array(y_test_reshaped)

# Build the LSTM model with additional layers and dropout
lstm_GAM = Sequential()
lstm_GAM.add(LSTM(128, input_shape=(sequence_length, len(input_features)), return_sequences=True))
lstm_GAM.add(Dropout(0.2))
lstm_GAM.add(LSTM(64, return_sequences=True))
lstm_GAM.add(Dropout(0.2))
lstm_GAM.add(LSTM(32))
lstm_GAM.add(Dropout(0.2))
lstm_GAM.add(Dense(1, activation='sigmoid'))
lstm_GAM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
lstm_GAM.fit(X_train_reshaped, y_train_reshaped, epochs=5, batch_size=32, validation_split=0.2, callbacks=[early_stop])

lstm_GAM.save('lstm_GAM.h5')

# Evaluate the model on the testing data
loss, accuracy = lstm_GAM.evaluate(X_test_reshaped, y_test_reshaped)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# ... (previous code remains the same)

# Evaluate the model on the testing data
loss, accuracy = lstm_GAM.evaluate(X_test_reshaped, y_test_reshaped)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Make predictions on the testing data
y_pred_prob = lstm_GAM.predict(X_test_reshaped)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Calculate the confusion matrix
cm = confusion_matrix(y_test_reshaped, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Attack'])
plt.yticks(tick_marks, ['Normal', 'Attack'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Add numbers to the confusion matrix
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# Assuming you have the following variables defined:
# test_data, sequence_length, y_pred

# Calculate the number of rows to be removed based on the sequence length
offset = sequence_length - 1  # This is the starting index for sequences

# Ensure that you're considering the correct number of elements from the end
test_data_with_predictions = test_data.iloc[offset:offset + len(y_pred)].copy()

# Now add the predictions
test_data_with_predictions['Predicted Attack'] = y_pred

# Check to ensure the DataFrame and predictions align
assert len(test_data_with_predictions) == len(y_pred), "The lengths do not match."

# Now, test_data_with_predictions includes the predictions aligned with the data

# Filter for rows where actual attack is 1 and predicted attack is 0
false_negatives = test_data_with_predictions[(test_data_with_predictions['Attack'] == 1) & (test_data_with_predictions['Predicted Attack'] == 0)]

# Display these rows
false_negatives

### LSTM without GAM

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

# Split the data into training and testing sets
train_data = manipulated_data_2[manipulated_data_2['# Date and time'].dt.year < 2021]
test_data = manipulated_data_2[manipulated_data_2['# Date and time'].dt.year == 2021]

# Define the input features and target
input_features = ['Wind speed (m/s)', 'Power (kW)', 'Rotor speed (RPM)']
target = 'Attack'

# Prepare the training and testing data
X_train = train_data[input_features].values
y_train = train_data[target].values
X_test = test_data[input_features].values
y_test = test_data[target].values

# Scale the input features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape the input data to include the sequence length
sequence_length = 10
X_train_reshaped = []
y_train_reshaped = []
for i in range(sequence_length, len(X_train_scaled)):
    X_train_reshaped.append(X_train_scaled[i - sequence_length:i])
    y_train_reshaped.append(y_train[i])  # Shift the labels by sequence_length
X_train_reshaped = np.array(X_train_reshaped)
y_train_reshaped = np.array(y_train_reshaped)

X_test_reshaped = []
y_test_reshaped = []
for i in range(sequence_length, len(X_test_scaled)):
    X_test_reshaped.append(X_test_scaled[i - sequence_length:i])
    y_test_reshaped.append(y_test[i])  # Shift the labels by sequence_length
X_test_reshaped = np.array(X_test_reshaped)
y_test_reshaped = np.array(y_test_reshaped)

# Build the LSTM model with additional layers and dropout
lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(sequence_length, len(input_features)), return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(32))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
lstm_model.fit(X_train_reshaped, y_train_reshaped, epochs=5, batch_size=32, validation_split=0.2, callbacks=[early_stop])

# Evaluate the model on the testing data
loss, accuracy = lstm_model.evaluate(X_test_reshaped, y_test_reshaped)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Evaluate the model on the testing data
loss, accuracy = lstm_model.evaluate(X_test_reshaped, y_test_reshaped)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Make predictions on the testing data
y_pred_prob = lstm_model.predict(X_test_reshaped)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Calculate the confusion matrix
cm = confusion_matrix(y_test_reshaped, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Attack'])
plt.yticks(tick_marks, ['Normal', 'Attack'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Add numbers to the confusion matrix
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

### Isolation forest with GAM

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import joblib

# Split the data into training and testing sets
train_data = manipulated_data_2[manipulated_data_2['# Date and time'].dt.year < 2021]
test_data = manipulated_data_2[manipulated_data_2['# Date and time'].dt.year == 2021]

# Define the input features and target
input_features = ['Wind speed (m/s)', 'Power_GAM_diff', 'Rotor_Speed_GAM_diff']
target = 'Attack'

# Prepare the training and testing data
X_train = train_data[input_features].values
y_train = train_data[target].values
X_test = test_data[input_features].values
y_test = test_data[target].values

# Scale the input features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an Isolation Forest model
IF_GAM = IsolationForest(n_estimators=200, contamination=0.05, random_state=42)

# Train the model
IF_GAM.fit(X_train_scaled)

joblib.dump(IF_GAM, 'iforest_GAM.pkl')

# Make predictions on the training and testing data
y_train_pred = IF_GAM.predict(X_train_scaled)
y_test_pred = IF_GAM.predict(X_test_scaled)

# Convert the predictions to binary labels (-1 for anomalies, 1 for normal)
y_train_pred = np.where(y_train_pred == -1, 1, 0)
y_test_pred = np.where(y_test_pred == -1, 1, 0)

# Evaluate the model on the training data
train_accuracy = (y_train_pred == y_train).mean()
print(f'Training Accuracy: {train_accuracy:.4f}')

# Evaluate the model on the testing data
test_accuracy = (y_test_pred == y_test).mean()
print(f'Testing Accuracy: {test_accuracy:.4f}')

In [ ]:
# Make predictions on the testing data
y_test_pred = IF_GAM.predict(X_test_scaled)

# Convert the predictions to binary labels (-1 for anomalies, 1 for normal)
y_test_pred = np.where(y_test_pred == -1, 1, 0)

# Evaluate the model on the testing data
test_accuracy = (y_test_pred == y_test).mean()
print(f'Testing Accuracy: {test_accuracy:.4f}')

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Anomaly'], rotation=45)
plt.yticks(tick_marks, ['Normal', 'Anomaly'])

# Add numbers to the confusion matrix
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

### Isolation forest without GAM

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

# Split the data into training and testing sets
train_data = manipulated_data_2[manipulated_data_2['# Date and time'].dt.year < 2021]
test_data = manipulated_data_2[manipulated_data_2['# Date and time'].dt.year == 2021]

# Define the input features and target
input_features = ['Wind speed (m/s)', 'Power (kW)', 'Rotor speed (RPM)']
target = 'Attack'

# Prepare the training and testing data
X_train = train_data[input_features].values
y_train = train_data[target].values
X_test = test_data[input_features].values
y_test = test_data[target].values

# Scale the input features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an Isolation Forest model
IF_model = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)

# Train the model
IF_model.fit(X_train_scaled)

# Make predictions on the training and testing data
y_train_pred = IF_model.predict(X_train_scaled)
y_test_pred = IF_model.predict(X_test_scaled)

# Convert the predictions to binary labels (-1 for anomalies, 1 for normal)
y_train_pred = np.where(y_train_pred == -1, 1, 0)
y_test_pred = np.where(y_test_pred == -1, 1, 0)

# Evaluate the model on the training data
train_accuracy = (y_train_pred == y_train).mean()
print(f'Training Accuracy: {train_accuracy:.4f}')

# Evaluate the model on the testing data
test_accuracy = (y_test_pred == y_test).mean()
print(f'Testing Accuracy: {test_accuracy:.4f}')

In [ ]:
# Make predictions on the testing data
y_test_pred = IF_model.predict(X_test_scaled)

# Convert the predictions to binary labels (-1 for anomalies, 1 for normal)
y_test_pred = np.where(y_test_pred == -1, 1, 0)

# Evaluate the model on the testing data
test_accuracy = (y_test_pred == y_test).mean()
print(f'Testing Accuracy: {test_accuracy:.4f}')

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Anomaly'], rotation=45)
plt.yticks(tick_marks, ['Normal', 'Anomaly'])

# Add numbers to the confusion matrix
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()